In [1]:
import networkx as nx
import json
import numpy as np
import random 
import pandas as pd
import sys
import itertools
import pickle
import time
import seaborn as sns
import matplotlib.pyplot as plt 

# Github Link: https://github.com/zmx-bruce/STAT605_GroupProject

## Code to load in data

In [6]:
label=pd.read_csv("reddit_threads/reddit_target.csv")
with open("reddit_threads/reddit_edges.json", 'rb') as f:
    data= json.load(f)
print(f"data contain {len(data)} networks")

data contain 203088 networks


### An example of loaded network

In [5]:
G=nx.from_edgelist(data['0'])
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 11
Number of edges: 10
Average degree:   1.8182


## Descriptions of data and variables

Our datasets contain 203088 networks, each network is labeled with 0 or 1.
This network classification dataset does not like normal matrix form datasets that have multiple observations and variables. Here our observations are networks. We could take network statistics as our variables.

Based on the descriptions of datasets, network number of nodes ranges from 11 to 97, network density ranges from 0.021 to 0.382, network diameter ranges from 2 to 27.

## Statistical methods

Our major statistical methods is hypothesis testing. To be more specific, we are going to use two sample hypothesis testing to test whether networks with different labels have different structure in terms of 3-node connected subgraph density.


## Computational steps 

Here we give an example of our computation in one network

1. We first define our 3-node subgraph structure.
2. Then we choose 3 from n number of nodes of a network, and check if these 3 nodes have the same structure with our target subgraph.

The example suggests that it takes about 2 second to complete the calculation on one single network (time depends on number of nodes). So with more than 200k networks in total, even it only takes 0.3 second for one network on average, in total it will take more than 15 hours to run on a local computer. 

So we plan to split job into 1000 parallel tasks on CHTC, for each task we calculate 200 networks. Ideally, this would save our time for 1000 times. It only takes 1 minute to complete all the calculation. 

In [14]:
motifs = {
'M3_1': nx.Graph([(1,2),(2,3)]),
'M3_2': nx.Graph([(1,2),(1,3),(2,3)]),}

G=nx.from_edgelist(data['30'])

start=time.time()
m3_1=0;m3_2=0
for sub_nodes in itertools.combinations(G.nodes(),3):# choose 3 from n, where n is number of nodes of network
    subg = G.subgraph(sub_nodes)
    if nx.is_connected(subg):
        if nx.is_isomorphic(subg, motifs['M3_1']):
            m3_1+=1
        if nx.is_isomorphic(subg, motifs['M3_2']):
            m3_2+=1
print("number of type_1 3-node subgraph(open triangular):",m3_1)
print("number of type_2 3-node subgraph(closed triangular):",m3_2)
end=time.time()
print("calculation time:",end-start)


number of type_1 3-node subgraph(open triangular): 1101
number of type_2 3-node subgraph(closed triangular): 1
calculation time: 1.9901082515716553
